In [1]:
import pandas as pd
import numpy as np
import itertools
from matplotlib import pyplot
# forecast monthly births with random forest
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
series = read_csv('datacount7.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
train_end_date = '2023-06-23'
train_end_date = pd.to_datetime(train_end_date)
train = series.loc[series.index < train_end_date]
test = series.loc[series.index >= train_end_date]
trainvalues = train.values
trainvalues = trainvalues.reshape(-1, 1)
testvalues = test.values
testvalues = testvalues.reshape(-1, 1)

In [3]:
def series_to_supervised(data,n_in = 1,n_out = 1,dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
         cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        return agg.values

In [4]:
traindata = series_to_supervised(trainvalues, n_in=8)
testdata = series_to_supervised(testvalues, n_in=8)

In [5]:
def random_forest_forecast(train, testX):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = RandomForestRegressor(n_estimators=n_estimators,max_depth = max_depth,min_samples_leaf = min_samples_leaf)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	return yhat[0]

In [6]:
def walk_forward_validation(traindata,testdata, n_test,n_estimators,max_depth,min_samples_leaf):
    predictions = list()
    history = [x for x in traindata]
    for i in range(len(testdata)):
        # split test row into input and output columns
        testX, testy = testdata[i, :-1], testdata[i, -1]
        # fit model on history and make a prediction
        yhat = random_forest_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(testdata[i])
    # estimate prediction error
    error = mean_absolute_error(testdata[:, -1], predictions)
    return error, testdata[:, -1], predictions

In [10]:
n_estimators = 31

In [11]:
max_depth = 4

In [12]:
min_samples_leaf=10

In [13]:
mae, y, yhat = walk_forward_validation(traindata,testdata, 31,n_estimators=n_estimators,max_depth = max_depth,min_samples_leaf = min_samples_leaf)

In [14]:
y

array([293., 230., 739., 679., 641., 592., 553., 275., 235., 690., 711.,
       633., 549., 527., 329., 217., 689., 684., 613., 539., 508., 302.,
       230., 687., 678., 625., 572., 535., 326., 248., 671.])

In [15]:
yhat

[330.26866785331214,
 606.5982998495477,
 671.838621180793,
 654.6304858124269,
 629.4605185812259,
 547.7217328790846,
 519.490492095028,
 316.1648810986384,
 232.7495316569506,
 674.2857506110895,
 653.3402111506489,
 635.2051474365574,
 554.748043528052,
 530.4165747263604,
 294.43230550895765,
 231.73680753306016,
 680.2291983634177,
 641.3959027457558,
 637.5804836139929,
 540.2838029747735,
 513.3258720654852,
 333.3273743319599,
 215.5130972040875,
 660.3432251634292,
 628.3424667205251,
 623.1997078058205,
 541.6935140683398,
 522.6752234302171,
 287.57409983198295,
 228.21554765124657,
 684.221421955327]

In [18]:
sumall = sum(y)

In [19]:
gap = sum(abs(yhat-y))

In [20]:
gap

1092.7457445060395

In [21]:
sumall

15800.0

In [22]:
gap/sumall

0.0691611230700025